# Pipe 공정 순서배열에 따른 소요시간 최적화 방법론

엑셀 데이터를 기반으로 제품 공정의 스케줄 최적화를 위한 Python 알고리즘을 개발하려고 합니다. 엑셀 데이터는 각 제품의 공정 단계(1단계부터 6단계까지)와 해당 단계의 평균 작업 시간을 포함하고 있습니다.

데이터 구조:

제품그룹: 제품 그룹을 나타내는 고유 식별자.
step1 평균 작업 시간(분) ~ step6 평균 작업 시간(분): 각 공정 단계의 평균 작업 시간.

제품개수: 각 제품 그룹별로 생산해야 할 제품의 수.

공정 조건:
각 제품은 1단계부터 6단계까지 순차적으로 공정을 진행해야 합니다.
각 공정 단계는 병렬 처리가 가능하지만, 다음과 같은 기계(호기) 제한이 있습니다:
1단계, 2단계, 5단계, 6단계: 각 단계당 1호기.
3단계: 3호기.
4단계: 4호기.
각 호기는 동시에 하나의 제품만을 처리할 수 있습니다.
제품의 공정 순서를 최적화하여 총 작업 소요 시간을 최소화하고, 대기 시간을 줄이고자 합니다.

예시 상황:
일부 제품은 1~5단계는 빠르게 완료되지만 6단계가 오래 걸릴 수 있습니다. 이러한 제품을 우선적으로 처리하여 전체 대기 시간을 줄이고자 합니다.
예를 들어, 제품 15가 3단계를 빠르게 완료했지만 제품 23이 4단계에서 오래 걸려 다음 단계로 넘어가는 데 지연이 발생할 수 있습니다. 이러한 대기 시간을 최소화하기 위해 제품의 공정 시작 순서를 최적화해야 합니다.

목표:
모든 제품의 공정을 완료했을 때의 총 작업 소요 시간과 최적화된 제품 순서를 도출하는 Python 코드를 작성합니다.

In [ ]:
# Pipe 공정의 자료구조는 모두 queue 구조를 따릅니다.

## 데이터 확인
1.   Raw 데이터
2.   GT값 (휴리스틱라벨링)
3.   정규분포에 따른 데이터 필터링
4.   our 데이터 전처리 시스템

*   같은 파이프 종류를 그룹별로 함께 작업하는 경우
*   전체 파이프에 대해 하나하나 따로 작업하는 경우



In [37]:
import pandas as pd
#1
#file_path = 'pipe_final.xlsx'
#2
#file_path = 'heuristic_data.xlsx'
#3
#file_path = 'filter_pipe_final_241007.xlsx'
#4
file_path = 'updated_total.xlsx'
#5
#file_path = 'updated_total_filter.xlsx'
df = pd.read_excel(file_path).dropna()
df

,제품군,외경/폭,두께,등록 길이,등록 중량,step1 최종 작업 시간,step2 최종 작업 시간,step3 최종 작업 시간,step4 최종 작업 시간,step5 최종 작업 시간,step6 최종 작업 시간
0,23A012-01,610.0,23.83,6.0,2.067,9.7,10.2,10.6,4.2,14.0,22.3
1,23A012-02,914.0,12.70,11.8,3.332,8.9,9.6,8.9,8.9,11.0,13.9
2,23A012-03,914.0,12.70,11.5,3.247,8.5,9.5,7.7,7.2,3.5,12.5
3,23A012-04,914.0,12.70,6.1,1.722,8.9,9.2,10.6,10.5,11.4,20.5
4,23A029-19,914.0,34.00,11.0,8.117,10.7,11.5,10.4,10.3,16.5,21.8
...,...,...,...,...,...,...,...,...,...,...,...
121,23Y020-04,864.0,9.50,6.0,1.202,5.4,9.9,10.5,9.4,14.0,21.6
122,23Y022-01,762.0,9.50,6.0,1.058,6.5,10.2,8.1,11.4,20.7,17.9
123,23Y023-01,762.0,25.40,11.8,5.445,11.6,11.7,7.3,9.4,18.5,10.4
124,23Y023-02,914.0,30.18,11.8,7.762,13.2,12.8,8.4,8.1,8.0,18.0


In [32]:
"""
# 4,5 데이터에 해당함
for idx, row in df.iterrows():
    product_group = row['제품군']
    processing_times = {
        'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
        'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
        'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
        'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
        'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
        'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
    }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })

--------------------------------------------------------------------------------------
# 1,2,3 데이터에 해당함
    for idx, row in df.iterrows():
        product_group = row['제품번호']
        processing_times_dict = {
            'step1': int(round(row['step1 작업 시간(분)'] * SCALE)),
            'step2': int(round(row['step2 작업 시간(분)'] * SCALE)),
            'step3': int(round(row['step3 작업 시간(분)'] * SCALE)),
            'step4': int(round(row['step4 작업 시간(분)'] * SCALE)),
            'step5': int(round(row['step5 작업 시간(분)'] * SCALE)),
            'step6': int(round(row['step6 작업 시간(분)'] * SCALE)),
        }
        units.append({
            'unit_id': f"{product_group}",
            'product_group': product_group,
            'processing_times': processing_times_dict,
        })
"""


'\n# 4,5 데이터에 해당함\nfor idx, row in df.iterrows():\n    product_group = row[\'제품군\']\n    processing_times = {\n        \'step1\': int(round(row[\'step1 최종 작업 시간\'] * SCALE)),\n        \'step2\': int(round(row[\'step2 최종 작업 시간\'] * SCALE)),\n        \'step3\': int(round(row[\'step3 최종 작업 시간\'] * SCALE)),\n        \'step4\': int(round(row[\'step4 최종 작업 시간\'] * SCALE)),\n        \'step5\': int(round(row[\'step5 최종 작업 시간\'] * SCALE)),\n        \'step6\': int(round(row[\'step6 최종 작업 시간\'] * SCALE)),\n    }\n    units.append({\n        \'unit_id\': f"{product_group}",\n        \'product_group\': product_group,\n        \'processing_times\': processing_times,\n    })\n\n--------------------------------------------------------------------------------------\n# 1,2,3 데이터에 해당함\n    for idx, row in df.iterrows():\n        product_group = row[\'제품번호\']\n        processing_times_dict = {\n            \'step1\': int(round(row[\'step1 작업 시간(분)\'] * SCALE)),\n            \'step2\': int(round(row[\'step

## Heapq 자료구조의 그리디 탐색 배열 정렬

# 1번경우를 고려한 최적화 방법론

In [33]:
import pandas as pd
import heapq
from collections import deque

# 제품 클래스 정의
class Product:
    def __init__(self, id, processing_times):
        self.id = id
        self.processing_times = processing_times  # 각 단계별 처리 시간 리스트 (1단계부터 6단계까지)
        self.current_step = 0  # 현재 단계 (0부터 시작)
        self.available_time = 0  # 제품이 다음 처리를 받을 수 있는 시간
        self.start_times = []  # 각 단계별 시작 시간 기록

    def __repr__(self):
        return f"Product(id={self.id})"

# 기계 클래스 정의
class Machine:
    def __init__(self, id, step):
        self.id = id
        self.step = step
        self.available_time = 0  # 기계가 사용 가능해지는 시간

# 이벤트 클래스 정의
class Event:
    def __init__(self, time, product, machine, event_type):
        self.time = time
        self.product = product
        self.machine = machine
        self.event_type = event_type  # 이벤트 타입 ('finish_processing' 등)

    def __lt__(self, other):
        return self.time < other.time

def schedule_products(products):
    # 각 단계별 대기열(deque) 초기화
    step_queues = {step: deque() for step in range(1, 7)}

    # 각 단계별 기계 초기화
    machines = []
    machines_by_step = {step: [] for step in range(1, 7)}
    # 1, 2, 5, 6단계는 각 1대의 기계
    for step in [1, 2, 5, 6]:
        machine = Machine(id=f"Machine_{step}_1", step=step)
        machines.append(machine)
        machines_by_step[step].append(machine)
    # 3단계는 3대의 기계
    for i in range(1, 4):
        machine = Machine(id=f"Machine_3_{i}", step=3)
        machines.append(machine)
        machines_by_step[3].append(machine)
    # 4단계는 4대의 기계
    for i in range(1, 5):
        machine = Machine(id=f"Machine_4_{i}", step=4)
        machines.append(machine)
        machines_by_step[4].append(machine)

    # 이벤트 큐 초기화
    event_queue = []

    # 6단계의 처리 시간이 긴 순서대로 제품 정렬 (처리 시간이 긴 제품을 우선적으로 처리)
    products.sort(key=lambda p: p.processing_times[5], reverse=True)

    # 제품들을 1단계 대기열에 추가
    for product in products:
        step_queues[1].append(product)

    # 1단계에서 제품을 기계에 할당
    assign_products_to_machines(step_queues, machines_by_step, event_queue, current_time=0)

    # 시뮬레이션 루프 시작
    while event_queue:
        # 다음 이벤트 가져오기
        event = heapq.heappop(event_queue)
        current_time = event.time
        product = event.product
        machine = event.machine

        # 기계의 사용 가능 시간 업데이트
        machine.available_time = current_time

        # 제품의 상태 업데이트
        product.available_time = current_time
        product.current_step += 1  # 다음 단계로 이동

        # 시작 시간 기록
        product.start_times.append((machine.step, current_time - product.processing_times[machine.step - 1]))

        if product.current_step > 6:
            # 모든 단계 완료
            continue

        # 제품을 다음 단계 대기열에 추가
        next_step = product.current_step
        step_queues[next_step].append(product)

        # 다음 단계에서 제품을 기계에 할당 시도
        assign_products_to_machines(step_queues, machines_by_step, event_queue, current_time)

    # 총 작업 시간 계산 (makespan)
    makespan = max(product.available_time for product in products)

    # 1단계 시작 시간 기준으로 제품 순서 최적화
    optimized_sequence = sorted(products, key=lambda p: p.start_times[0][1])

    # 결과 출력
    print(f"총 작업 소요 시간: {makespan} 분")
    print("최적화된 제품 순서 (1단계 처리 순서):")
    for product in optimized_sequence:
        print(f"제품 ID: {product.id}")

def assign_products_to_machines(step_queues, machines_by_step, event_queue, current_time):
    # 각 단계별로 사용 가능한 기계에 제품 할당
    for step, machines in machines_by_step.items():
        queue = step_queues[step]
        for machine in machines:
            # 대기 중인 제품이 있고 기계가 사용 가능하면
            while queue and machine.available_time <= current_time:
                product = queue.popleft()
                # 시작 시간 계산
                start_time = max(product.available_time, machine.available_time)
                # 완료 시간 계산
                processing_time = product.processing_times[step - 1]
                finish_time = start_time + processing_time
                # 처리 완료 이벤트 생성
                event = Event(time=finish_time, product=product, machine=machine, event_type='finish_processing')
                heapq.heappush(event_queue, event)
                # 기계의 사용 가능 시간 업데이트
                machine.available_time = finish_time
                # 기계에 한 번에 하나의 제품만 할당하므로 루프 종료
                break

def main():
    # 엑셀 파일에서 데이터 읽기
    import pandas as pd

    # 예를 들어, 'data.xlsx' 파일에서 데이터 읽기
    df = pd.read_excel(file_path).dropna()

    SCALE = 1  # 필요에 따라 조정

    units = []
    for idx, row in df.iterrows():
      product_group = row['제품군']
      processing_times = {
          'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
          'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
          'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
          'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
          'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
          'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
      }
      units.append({
          'unit_id': f"{product_group}",
          'product_group': product_group,
          'processing_times': processing_times,
      })


    # Product 인스턴스 생성
    products = []
    for unit in units:
        id = unit['unit_id']
        processing_times_dict = unit['processing_times']
        # processing_times_dict를 리스트로 변환
        processing_times = [
            processing_times_dict['step1'],
            processing_times_dict['step2'],
            processing_times_dict['step3'],
            processing_times_dict['step4'],
            processing_times_dict['step5'],
            processing_times_dict['step6'],
        ]
        product = Product(id=id, processing_times=processing_times)
        products.append(product)

    # 스케줄링 알고리즘 실행
    schedule_products(products)

if __name__ == "__main__":
    main()


총 작업 소요 시간: 1385 분
최적화된 제품 순서 (1단계 처리 순서):
제품 ID: 23Y019-03
제품 ID: 23D017-06
제품 ID: 23D017-19
제품 ID: 23D021-11
제품 ID: 23A012-01
제품 ID: 23A029-19
제품 ID: 23D016-03
제품 ID: 23D017-42
제품 ID: 23D020-03
제품 ID: 23D020-04
제품 ID: 23D021-25
제품 ID: 23D024-03
제품 ID: 23Y016-02
제품 ID: 23Y020-04
제품 ID: 23C009-06
제품 ID: 23D016-06
제품 ID: 23D018-09
제품 ID: 23D021-23
제품 ID: 23D024-01
제품 ID: 23D025-01
제품 ID: 23E003-01
제품 ID: 23A012-04
제품 ID: 23D004-02
제품 ID: 23D013-02
제품 ID: 23D023-06
제품 ID: 23D024-02
제품 ID: 23E029-02
제품 ID: 23B004-15
제품 ID: 23D025-03
제품 ID: 23D025-05
제품 ID: 23E019-01
제품 ID: 23E020-02
제품 ID: 23Y019-04
제품 ID: 23E007-01
제품 ID: 23E018-08
제품 ID: 23Y016-05
제품 ID: 23Y022-01
제품 ID: 23Y023-02
제품 ID: 23D016-02
제품 ID: 23D017-20
제품 ID: 23D024-04
제품 ID: 23D026-01
제품 ID: 23D027-01
제품 ID: 23E017-06
제품 ID: 23D013-01
제품 ID: 23D017-24
제품 ID: 23D018-03
제품 ID: 23D021-04
제품 ID: 23D021-05
제품 ID: 23D021-27
제품 ID: 23D023-01
제품 ID: 23D023-03
제품 ID: 23E012-04
제품 ID: 23E029-03
제품 ID: 23Y018-01
제품 ID: 23Y023-03
제품 ID

# 2번 경우를 고려한 최적화방법

In [34]:
import pandas as pd
import heapq
from collections import deque

# 제품 클래스 정의
class Product:
    def __init__(self, id, processing_times):
        self.id = id
        self.processing_times = processing_times  # 각 단계별 처리 시간 리스트 (1단계부터 6단계까지)
        self.current_step = 0  # 현재 단계 (0부터 시작)
        self.available_time = 0  # 제품이 다음 처리를 받을 수 있는 시간
        self.start_times = []  # 각 단계별 시작 시간 기록

    def total_processing_time(self):
        return sum(self.processing_times)

    def __repr__(self):
        return f"Product(id={self.id})"

# 기계 클래스 정의
class Machine:
    def __init__(self, id, step):
        self.id = id
        self.step = step
        self.available_time = 0  # 기계가 사용 가능해지는 시간

# 이벤트 클래스 정의
class Event:
    def __init__(self, time, product, machine, event_type):
        self.time = time
        self.product = product
        self.machine = machine
        self.event_type = event_type  # 이벤트 타입 ('finish_processing' 등)

    def __lt__(self, other):
        return self.time < other.time

def schedule_products(products):
    # 각 단계별 대기열(deque) 초기화
    step_queues = {step: deque() for step in range(1, 7)}

    # 각 단계별 기계 초기화
    machines_by_step = {step: [] for step in range(1, 7)}
    # 1, 2, 5, 6단계는 각 1대의 기계
    for step in [1, 2, 5, 6]:
        machine = Machine(id=f"Machine_{step}_1", step=step)
        machines_by_step[step].append(machine)
    # 3단계는 3대의 기계
    for i in range(1, 4):
        machine = Machine(id=f"Machine_3_{i}", step=3)
        machines_by_step[3].append(machine)
    # 4단계는 4대의 기계
    for i in range(1, 5):
        machine = Machine(id=f"Machine_4_{i}", step=4)
        machines_by_step[4].append(machine)

    # 이벤트 큐 초기화
    event_queue = []

    # **그리디 알고리즘 적용 부분**
    # 제품들을 총 처리 시간이 긴 순서대로 정렬
    products.sort(key=lambda p: p.total_processing_time(), reverse=True)

    # 제품들을 1단계 대기열에 추가
    for product in products:
        step_queues[1].append(product)

    # 1단계에서 제품을 기계에 할당
    assign_products_to_machines(step_queues, machines_by_step, event_queue, current_time=0)

    # 시뮬레이션 루프 시작
    while event_queue:
        # 다음 이벤트 가져오기
        event = heapq.heappop(event_queue)
        current_time = event.time
        product = event.product
        machine = event.machine

        # 기계의 사용 가능 시간 업데이트
        machine.available_time = current_time

        # 제품의 상태 업데이트
        product.available_time = current_time
        product.current_step += 1  # 다음 단계로 이동

        # 시작 시간 기록
        product.start_times.append((machine.step, current_time - product.processing_times[machine.step - 1]))

        if product.current_step > 6:
            # 모든 단계 완료
            continue

        # 제품을 다음 단계 대기열에 추가
        next_step = product.current_step
        step_queues[next_step].append(product)

        # 다음 단계에서 제품을 기계에 할당 시도
        assign_products_to_machines(step_queues, machines_by_step, event_queue, current_time)

    # 총 작업 시간 계산 (makespan)
    makespan = max(product.available_time for product in products)

    # 제품들의 1단계 시작 시간을 기준으로 정렬하여 처리 순서 확인
    optimized_sequence = sorted(products, key=lambda p: p.start_times[0][1])

    # 결과 출력
    print(f"총 작업 소요 시간: {makespan} 분")
    print("최적화된 제품 순서 (1단계 처리 순서):")
    for product in optimized_sequence:
        print(f"제품 ID: {product.id}")

def assign_products_to_machines(step_queues, machines_by_step, event_queue, current_time):
    # 각 단계별로 사용 가능한 기계에 제품 할당
    for step, machines in machines_by_step.items():
        queue = step_queues[step]
        for machine in machines:
            # 대기 중인 제품이 있고 기계가 사용 가능하면
            while queue and machine.available_time <= current_time:
                product = queue.popleft()
                # 시작 시간 계산
                start_time = max(product.available_time, machine.available_time)
                # 완료 시간 계산
                processing_time = product.processing_times[step - 1]
                finish_time = start_time + processing_time
                # 처리 완료 이벤트 생성
                event = Event(time=finish_time, product=product, machine=machine, event_type='finish_processing')
                heapq.heappush(event_queue, event)
                # 기계의 사용 가능 시간 업데이트
                machine.available_time = finish_time
                # 기계에 한 번에 하나의 제품만 할당하므로 루프 종료
                break

def main():
    # 엑셀 파일에서 데이터 읽기
    import pandas as pd

    # 예를 들어, 'data.xlsx' 파일에서 데이터 읽기
    df = pd.read_excel(file_path).dropna()

    SCALE = 1  # 필요에 따라 조정

    products = []
    for idx, row in df.iterrows():
        product_id = row['제품군']
        processing_times_dict = {
            'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
            'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
            'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
            'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
            'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
            'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
        }
        # 각 제품을 개별적으로 처리
        processing_times = [
            processing_times_dict['step1'],
            processing_times_dict['step2'],
            processing_times_dict['step3'],
            processing_times_dict['step4'],
            processing_times_dict['step5'],
            processing_times_dict['step6'],
        ]
        product = Product(id=product_id, processing_times=processing_times)
        products.append(product)

    # 스케줄링 알고리즘 실행
    schedule_products(products)

if __name__ == "__main__":
    main()


총 작업 소요 시간: 1426 분
최적화된 제품 순서 (1단계 처리 순서):
제품 ID: 23Y019-03
제품 ID: 23D024-02
제품 ID: 23D021-23
제품 ID: 23A029-19
제품 ID: 23D016-02
제품 ID: 23D016-03
제품 ID: 23D021-11
제품 ID: 23D016-06
제품 ID: 23D023-06
제품 ID: 23D024-05
제품 ID: 23D004-02
제품 ID: 23D013-02
제품 ID: 23E003-01
제품 ID: 23B004-15
제품 ID: 23D024-01
제품 ID: 23E017-06
제품 ID: 23D021-25
제품 ID: 23Y018-01
제품 ID: 23Y022-01
제품 ID: 23D017-06
제품 ID: 23D020-03
제품 ID: 23D021-05
제품 ID: 23D025-03
제품 ID: 23E017-08
제품 ID: 23E020-02
제품 ID: 23D024-03
제품 ID: 23D024-04
제품 ID: 23D025-05
제품 ID: 23E019-01
제품 ID: 23Y019-04
제품 ID: 23A012-01
제품 ID: 23D020-04
제품 ID: 23D025-01
제품 ID: 23Y016-02
제품 ID: 23A012-04
제품 ID: 23C009-06
제품 ID: 23D017-19
제품 ID: 23D017-20
제품 ID: 23D017-42
제품 ID: 23D018-09
제품 ID: 23D023-03
제품 ID: 23E029-02
제품 ID: 23Y020-04
제품 ID: 23E029-03
제품 ID: 23Y016-05
제품 ID: 23D021-27
제품 ID: 23E018-08
제품 ID: 23Y023-01
제품 ID: 23Y023-02
제품 ID: 23Y016-08
제품 ID: 23D007-02
제품 ID: 23D023-04
제품 ID: 23D021-04
제품 ID: 23E020-01
제품 ID: 23D018-06
제품 ID: 23E007-01
제품 ID

#Pipe의 공정 순서 배열 csv 데이터를 바탕으로 공정 전체 소요시간을 계산하는 코드

In [38]:
import pandas as pd
from ortools.sat.python import cp_model

# 엑셀 데이터를 읽어옵니다.
df = pd.read_excel(file_path).fillna(0)

# NaN 값이 있는 행을 제거합니다.
df.dropna(inplace=True)

# 제품 데이터를 단위 제품으로 만듭니다.
units = []
# **시간 단위를 정수로 변환하기 위한 스케일링 팩터를 정의합니다.**
SCALE = 1000  # 소수점 이하 3자리까지 표현


for idx, row in df.iterrows():
    product_group = row['제품군']
    processing_times = {
            'step1': int(round(row['step1 최종 작업 시간'] * SCALE)),
            'step2': int(round(row['step2 최종 작업 시간'] * SCALE)),
            'step3': int(round(row['step3 최종 작업 시간'] * SCALE)),
            'step4': int(round(row['step4 최종 작업 시간'] * SCALE)),
            'step5': int(round(row['step5 최종 작업 시간'] * SCALE)),
            'step6': int(round(row['step6 최종 작업 시간'] * SCALE)),
        }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })

# 스케줄링 문제를 정의합니다.
model = cp_model.CpModel()

steps = ['step1', 'step2', 'step3', 'step4', 'step5', 'step6']
machines_per_step = {
    'step1': 1,
    'step2': 1,
    'step3': 3,
    'step4': 4,
    'step5': 1,
    'step6': 1,
}

# 각 단위 제품과 단계에 대한 변수들을 정의합니다.
horizon = sum([u['processing_times'][s] for u in units for s in steps])
unit_step_intervals = {}

for unit in units:
    unit_id = unit['unit_id']
    processing_times = unit['processing_times']
    for step in steps:
        duration = int(processing_times[step])
        start_var = model.NewIntVar(0, horizon, f'start_{unit_id}_{step}')
        end_var = model.NewIntVar(0, horizon, f'end_{unit_id}_{step}')
        interval_var = model.NewIntervalVar(start_var, duration, end_var, f'interval_{unit_id}_{step}')
        unit_step_intervals[(unit_id, step)] = interval_var

# 단계별 선행 조건을 추가합니다.
for unit in units:
    unit_id = unit['unit_id']
    for s in range(len(steps) - 1):
        step_current = steps[s]
        step_next = steps[s + 1]
        end_current = unit_step_intervals[(unit_id, step_current)].EndExpr()
        start_next = unit_step_intervals[(unit_id, step_next)].StartExpr()
        model.Add(end_current <= start_next)

# 각 단계의 기계 용량 제약 조건을 추가합니다.
for step in steps:
    intervals = []
    demands = []
    for unit in units:
        unit_id = unit['unit_id']
        intervals.append(unit_step_intervals[(unit_id, step)])
        demands.append(1)
    machine_capacity = machines_per_step[step]
    model.AddCumulative(intervals, demands, machine_capacity)

# 총 작업 시간을 최소화하는 목표를 설정합니다.
makespan = model.NewIntVar(0, horizon, 'makespan')
end_times = []
for unit in units:
    unit_id = unit['unit_id']
    end_times.append(unit_step_intervals[(unit_id, 'step6')].EndExpr())
model.AddMaxEquality(makespan, end_times)
model.Minimize(makespan)

# 모델을 해결합니다.
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

# 결과를 출력합니다.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"총 작업 시간: {solver.ObjectiveValue()} 분")
    schedule = []
    for unit in units:
        unit_id = unit['unit_id']
        unit_schedule = {'unit_id': unit_id, 'steps': {}}
        for step in steps:
            start = solver.Value(unit_step_intervals[(unit_id, step)].StartExpr())
            end = solver.Value(unit_step_intervals[(unit_id, step)].EndExpr())
            unit_schedule['steps'][step] = {'start': start, 'end': end}
        schedule.append(unit_schedule)
    schedule.sort(key=lambda x: x['steps']['step1']['start'])
    for unit_schedule in schedule:
        unit_id = unit_schedule['unit_id']
        print(f"제품 그룹: {unit_id}")
        for step in steps:
            start = unit_schedule['steps'][step]['start']
            end = unit_schedule['steps'][step]['end']
            print(f"  {step}: {start} 분에 시작, {end} 분에 종료")
else:
    print("해결 가능한 솔루션을 찾지 못했습니다.")


총 작업 시간: 5627800.0 분
제품 그룹: 23D017-30
  step1: 0 분에 시작, 15200 분에 종료
  step2: 19800 분에 시작, 31200 분에 종료
  step3: 31200 분에 시작, 35100 분에 종료
  step4: 35100 분에 시작, 46700 분에 종료
  step5: 53400 분에 시작, 53500 분에 종료
  step6: 1658500 분에 시작, 3158000 분에 종료
제품 그룹: 23D021-14
  step1: 0 분에 시작, 0 분에 종료
  step2: 139400 분에 시작, 148600 분에 종료
  step3: 148600 분에 시작, 158100 분에 종료
  step4: 158100 분에 시작, 165500 분에 종료
  step5: 592100 분에 시작, 602700 분에 종료
  step6: 4564600 분에 시작, 4583500 분에 종료
제품 그룹: 23D021-16
  step1: 0 분에 시작, 0 분에 종료
  step2: 925600 분에 시작, 935900 분에 종료
  step3: 935900 분에 시작, 944100 분에 종료
  step4: 944100 분에 시작, 952700 분에 종료
  step5: 959400 분에 시작, 966600 분에 종료
  step6: 5145900 분에 시작, 5161400 분에 종료
제품 그룹: 23Y016-04
  step1: 0 분에 시작, 0 분에 종료
  step2: 10200 분에 시작, 19800 분에 종료
  step3: 19800 분에 시작, 30400 분에 종료
  step4: 30400 분에 시작, 41000 분에 종료
  step5: 53500 분에 시작, 60500 분에 종료
  step6: 4676100 분에 시작, 4694200 분에 종료
제품 그룹: 23Y020-05
  step1: 0 분에 시작, 0 분에 종료
  step2: 0 분에 시작, 10200 분에 종료
  step3: 10200 분에 

In [ ]:
# 코드스니펫
"""
'step1 작업 시간(분)', 'step2 작업 시간(분)',
         'step3 작업 시간(분)', 'step4 작업 시간(분)',
         'step5 작업 시간(분)', 'step6 작업 시간(분)'

for idx, row in df.iterrows():
    product_group = row['제품번호']
    processing_times = {
        'step1': int(round(row['step1 작업 시간(분)'] * SCALE)),
        'step2': int(round(row['step2 작업 시간(분)'] * SCALE)),
        'step3': int(round(row['step3 작업 시간(분)'] * SCALE)),
        'step4': int(round(row['step4 작업 시간(분)'] * SCALE)),
        'step5': int(round(row['step5 작업 시간(분)'] * SCALE)),
        'step6': int(round(row['step6 작업 시간(분)'] * SCALE)),
    }
    units.append({
        'unit_id': f"{product_group}",
        'product_group': product_group,
        'processing_times': processing_times,
    })

In [16]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.2

In [17]:
df

,제품번호,외경/폭(mm),두께(mm),등록 길이(m),등록 중량(ton),step1 작업 시간(분),step2 작업 시간(분),step3 작업 시간(분),step4 작업 시간(분),step5 작업 시간(분),step6 작업 시간(분)
0,23A003-09-003,1676.0,28.58,6.0,6.967,0.0,0.0,0.0,0.0,0.0,0.0
1,23A003-09-004,1676.0,28.58,6.0,6.967,0.0,0.0,0.0,0.0,0.0,64.2
2,23A003-18-008,1626.0,30.18,6.0,7.127,0.0,0.0,0.0,0.0,0.0,0.0
3,23A003-19-003,1676.0,23.83,6.0,5.826,0.0,0.0,0.0,0.0,0.0,0.0
4,23A003-29-004,1626.0,30.18,6.0,7.127,0.0,0.0,0.0,0.0,0.0,96.9
...,...,...,...,...,...,...,...,...,...,...,...
892,23Y023-02-010,914.0,30.18,11.8,7.762,67.4,0.0,0.0,0.0,13.9,0.0
893,23Y023-02-012,914.0,30.18,11.8,7.762,0.0,0.0,0.0,0.0,14.5,22.5
894,23Y023-02-016,914.0,30.18,11.8,7.762,0.0,0.0,0.0,0.0,12.6,0.0
895,23Y023-02-017,914.0,30.18,11.8,7.762,0.0,26.4,0.0,0.0,13.9,22.8
